In [1]:
!pwd


/Users/mannalytics/Documents/Notebooks/pandas_research


# Decorators

In [194]:

def pivot_ts(*args, **kw):
    """
    *args should just be a dataframe
    **kw should just be the standard args for a dataframe
    returns a pivoted dataframe that changes the domain of timedeltas to seconds
    and then back again after the pivot
    """
    df = args[0]
    df['secs'] = df[kw['values']].dt.total_seconds()
    df2 = pd.pivot_table(df, index=kw['index'], columns=kw['columns'], values='secs')
    return df2.apply(pd.to_timedelta, unit='s')

def ts_agg_dec(ds_cols_l):
    """this decorator takes a list of columns which are timedeltas in a dataframe,
    it will change the timedeltas to seconds, apply the decorated function to the dataframe,
    and then translate the new dataframe back into timedelta's 
    - actually it currently requires that whole dataframe is comprised of timedeltas.
    """
    def ts_dec(f):
        def wrapper(*args, **kw):
            df = args[0].copy()

            # translate timedelta's to seconds
            for c in ds_cols_l:
                df[c] = df[c].dt.total_seconds()
            df2 = f(df, **kw) 
            df2 = df2.apply(pd.to_timedelta, unit='s')
            return df2
        return wrapper
    return ts_dec

        

In [187]:
# this works
@ts_agg_dec(["Calc_time"])
def test(df, index, columns, values):
    return pd.pivot_table(df, index=index, columns=columns, values=values)
    

In [188]:
# so does this
@ts_agg_dec(["Calc_time"])
def test2(*args, **kw):
    return pd.pivot_table(*args, **kw)
    

In [189]:
# and so does this!
test2(jr121,index='Team', columns='lap',values='Calc_time')

lap,1,2,3
Team,,,
Beavers,00:04:45,00:04:10,00:05:06
Caribou,00:04:59,00:05:14,00:03:45
Grizzly Bears,00:03:55,00:05:16,00:03:26
Lynxes,00:03:50,00:04:15,00:04:02
Moose,00:07:45,00:10:17,NaT
Otters,00:05:00,00:08:03,00:05:17


In [190]:
# even this does
ts_agg_dec(['Calc_time'])(pd.pivot_table)(jr121,index='Team', columns='lap',values='Calc_time')

lap,1,2,3
Team,,,
Beavers,00:04:45,00:04:10,00:05:06
Caribou,00:04:59,00:05:14,00:03:45
Grizzly Bears,00:03:55,00:05:16,00:03:26
Lynxes,00:03:50,00:04:15,00:04:02
Moose,00:07:45,00:10:17,NaT
Otters,00:05:00,00:08:03,00:05:17


In [191]:
test(jr121,index='Team', columns='lap',values='Calc_time')

lap,1,2,3
Team,,,
Beavers,00:04:45,00:04:10,00:05:06
Caribou,00:04:59,00:05:14,00:03:45
Grizzly Bears,00:03:55,00:05:16,00:03:26
Lynxes,00:03:50,00:04:15,00:04:02
Moose,00:07:45,00:10:17,NaT
Otters,00:05:00,00:08:03,00:05:17


In [168]:
jr121.dtypes

lap                      int64
FirstName               object
LastName                object
Category                object
Start_time              object
Watch_time     timedelta64[ns]
Calc_time      timedelta64[ns]
Team                    object
Offset_time    timedelta64[ns]
Offset_secs            float64
dtype: object

In [130]:
pivot_ts(jr121, index='Team', columns='lap', values='Offset_time')

lap,1,2,3
Team,,,
Beavers,00:00:55,00:05:05,00:10:11
Caribou,00:01:09,00:06:23,00:10:08
Grizzly Bears,00:00:05,00:05:21,00:08:47
Lynxes,00:00:00,00:04:15,00:08:17
Moose,00:03:55,00:14:12,NaT
Otters,00:01:10,00:09:13,00:14:30
